In [1]:
import pandas as pd
import seaborn as sns

from tqdm.notebook import tqdm
#nltk.download('punkt') #Uncomment if first run
#nltk.download('wordnet') #Uncomment if first run
#nltk.download('stopwords')
#nltk.download('omw-1.4')

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.lm import counter


In [ ]:
"""all_df = pd.read_csv('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/all_scraped_df_12_million_2020_01_01_v4')

all_df = all_df.drop(
    ['Unnamed: 0', '_type','url', 'renderedContent', 'id', 'replyCount', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
     'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags'],axis=1)

all_df = all_df.drop(
    ['retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links',
     'media', 'cashtags', 'card'], axis=1)

all_df = all_df.drop_duplicates('id')

all_df.to_csv('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/all_tweets_dates_user_v4.csv')"""


In [2]:
all_df = pd.read_csv('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/all_tweets_dates_user_v4.csv')
all_df

,Unnamed: 0,date,rawContent,user
0,0,2022-10-16 23:34:03+00:00,@dobbs_ddobell1 @hkuppy The problem is that th...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
1,1,2022-10-16 21:09:12+00:00,@silent_lasagna6 Unbelievable 😲. I think they ...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
2,2,2022-10-16 18:01:56+00:00,@DavidPepper People need to be reminded that t...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
3,3,2022-10-15 17:42:10+00:00,Thread. This is my deep fear as well. There ha...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
4,4,2022-10-15 15:15:24+00:00,@Energy_Tidbits @cafreeland Canadian governmen...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
...,...,...,...,...
10731641,14909120,2021-06-16 05:07:52+00:00,"@academy_share Sir, Jyothy Labs Ltd. Share cap...","{'_type': 'snscrape.modules.twitter.User', 'us..."
10731642,14909961,2021-02-03 04:10:45+00:00,@Rawlo__ @rtsmallstreams @share_stream @Stream...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
10731643,14910017,2021-01-25 14:43:02+00:00,Thank you everyone I met my goal of 200 follow...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
10731644,14910021,2021-01-25 05:26:00+00:00,@SharppzTTV @LDTStreamers @BlazedRTs @sme_rt @...,"{'_type': 'snscrape.modules.twitter.User', 'us..."


In [3]:
stop = set(stopwords.words('english'))
never_words = ['http', "n't", 'need', 'make', 'could, also', 'last', 'like', 'google', 'magicman', 'jacksonwang', "jacksonwang852", 'amp', 'que' 'love', 'water', 'call', 'via', 'would', 'know', 'even', 'say', 'thing', "'re" 'see', 'really', 'could', 'still', 'got', 'many', 'also', 'one', 'get', 'think', 'vs.', 'one', 'ever', 'ronfilipkowski']

In [4]:
def preprocess_tweets(element):
    lem=WordNetLemmatizer()
    words=[w for w in word_tokenize(element) if (w not in stop)]
    words=[lem.lemmatize(w.lower()) for w in words if len(w)>2]
    words=[word.lower() for word in words if word not in stop and word not in never_words]
    return words


all_df['preprocessedContent'] = all_df['rawContent'].apply(lambda x: preprocess_tweets(x))

all_df['preprocessedContent']

0           [dobbs_ddobell1, hkuppy, problem, middle, anym...
1           [silent_lasagna6, unbelievable, let, dobbs, pl...
2           [davidpepper, people, reminded, dobbs, decisio...
3           [thread, deep, fear, well, nearly, protest, do...
4           [energy_tidbits, cafreeland, canadian, governm...
                                  ...                        
10731641    [academy_share, sir, jyothy, lab, ltd., share,...
10731642    [rawlo__, rtsmallstreams, share_stream, stream...
10731643    [thank, everyone, met, goal, 200, follows, wow...
10731644    [sharppzttv, ldtstreamers, blazedrts, sme_rt, ...
10731645    [indikator, analisis, fundamental, umum, digun...
Name: preprocessedContent, Length: 10731646, dtype: object

In [7]:
preprocessed_tweet_data = all_df['preprocessedContent'].to_list()
preprocessed_tweet_data

[['dobbs_ddobell1',
  'hkuppy',
  'problem',
  'middle',
  'anymore',
  'right',
  'left',
  'extreme',
  'imagine',
  'world',
  'government',
  'campaigning',
  'funded',
  'tax',
  'dollar',
  'instead',
  'private',
  'donation',
  'might',
  'government',
  'without',
  'bias',
  'towards',
  'sugar',
  'daddy'],
 ['silent_lasagna6', 'unbelievable', 'let', 'dobbs', 'play', 'saying'],
 ['davidpepper',
  'people',
  'reminded',
  'dobbs',
  'decision',
  'overturning',
  'privacy',
  'right',
  'reproductive',
  'right',
  'abortion',
  'first',
  'private',
  'decision',
  'fall'],
 ['thread',
  'deep',
  'fear',
  'well',
  'nearly',
  'protest',
  'dobbs',
  'hoped',
  'hope',
  'private',
  'personal',
  'vote',
  'publicly',
  'protest'],
 ['energy_tidbits',
  'cafreeland',
  'canadian',
  'government',
  'look',
  'economically',
  'viable',
  'project',
  'way',
  'private',
  'industry',
  'move',
  'forward',
  'government',
  'caused',
  'supply',
  'crunch'],
 ['katlighti

In [9]:
import pickle

with open('all_preprocessed_tokens_v4.pkl', 'wb') as f:
    pickle.dump(preprocessed_tweet_data, f)

In [3]:
import pickle
with open('all_preprocessed_tokens_v4.pkl', 'rb') as f:
    text_data = pickle.load(f)

text_data[0:9]

[['dobbs_ddobell1',
  'hkuppy',
  'problem',
  'middle',
  'anymore',
  'right',
  'left',
  'extreme',
  'imagine',
  'world',
  'government',
  'campaigning',
  'funded',
  'tax',
  'dollar',
  'instead',
  'private',
  'donation',
  'might',
  'government',
  'without',
  'bias',
  'towards',
  'sugar',
  'daddy'],
 ['silent_lasagna6', 'unbelievable', 'let', 'dobbs', 'play', 'saying'],
 ['davidpepper',
  'people',
  'reminded',
  'dobbs',
  'decision',
  'overturning',
  'privacy',
  'right',
  'reproductive',
  'right',
  'abortion',
  'first',
  'private',
  'decision',
  'fall'],
 ['thread',
  'deep',
  'fear',
  'well',
  'nearly',
  'protest',
  'dobbs',
  'hoped',
  'hope',
  'private',
  'personal',
  'vote',
  'publicly',
  'protest'],
 ['energy_tidbits',
  'cafreeland',
  'canadian',
  'government',
  'look',
  'economically',
  'viable',
  'project',
  'way',
  'private',
  'industry',
  'move',
  'forward',
  'government',
  'caused',
  'supply',
  'crunch'],
 ['katlighti

In [1]:
import gensim.models

In [3]:
dic = gensim.corpora.Dictionary(text_data)
dic.save('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/gemsim_dic_for_LDA')
dic

In [2]:
dic = gensim.corpora.Dictionary.load('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/gemsim_dic_for_LDA')


In [4]:
from tqdm.notebook import tqdm
bow_corpus = []
for doc in tqdm(text_data):
    bow_corpus.append(dic.doc2bow(doc))
print(len(bow_corpus))

  0%|          | 0/10731646 [00:00<?, ?it/s]

10731646


In [ ]:

lda_model = gensim.models.LdaModel(bow_corpus,
                                   num_topics = 20,
                                   id2word = dic,
                                   passes = 5)


In [9]:
lda_model.print_topics()


[(0,
  '0.060*"short" + 0.053*"name" + 0.041*"join" + 0.038*"que" + 0.032*"start" + 0.028*"le" + 0.028*"fan" + 0.024*"probably" + 0.022*"dr." + 0.019*"fun"'),
 (1,
  '0.222*"roe" + 0.090*"abortion" + 0.063*"state" + 0.051*"law" + 0.042*"constitution" + 0.030*"amendment" + 0.019*"constitutional" + 0.019*"legal" + 0.019*"scotus" + 0.019*"case"'),
 (2,
  '0.208*"..." + 0.050*"home" + 0.036*"...." + 0.025*"comment" + 0.022*"article" + 0.022*"illegal" + 0.021*"coming" + 0.021*"open" + 0.017*"stay" + 0.017*"violation"'),
 (3,
  '0.061*"search" + 0.033*"pro" + 0.030*"overturn" + 0.027*"wait" + 0.026*"anti-abortion" + 0.026*"remember" + 0.026*"found" + 0.024*"sander" + 0.020*"favorite" + 0.019*"leave"'),
 (4,
  '0.504*"jackson" + 0.070*"court" + 0.036*"supreme" + 0.020*"justice" + 0.015*"experience" + 0.011*"set" + 0.010*"county" + 0.009*"city" + 0.008*"precedent" + 0.008*"rape"'),
 (5,
  '0.055*"today" + 0.045*"thank" + 0.037*"read" + 0.029*"something" + 0.027*"thanks" + 0.027*"end" + 0.024*"

In [6]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("model")
lda_model.save(temp_file)

In [7]:
import pyLDAvis.gensim_models
import pyLDAvis

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dic)
pyLDAvis.save_html(vis, '/home/karl/PycharmProjects/UCSF_dobbs_roe/lda4.html')

/home/karl/anaconda3/envs/UCSF_dobbs_roe/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/karl/anaconda3/envs/UCSF_dobbs_roe/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [ ]:
index_list = []
for i, lis in enumerate(lda_model[bow_corpus]):
    maxi = sorted(lis, key=lambda item: item[1], reverse=True)[0]
    if maxi[0] == 1 or maxi[0] == 19 and maxi[1] > 0.95:
        #print(lis)
        #print(i, maxi)
        index_list.append(i)
len(index_list)

In [13]:
import pandas as pd
all_df = pd.read_csv('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/all_tweets_dates_user_v4.csv')
all_df

,Unnamed: 0,date,rawContent,user
0,0,2022-10-16 23:34:03+00:00,@dobbs_ddobell1 @hkuppy The problem is that th...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
1,1,2022-10-16 21:09:12+00:00,@silent_lasagna6 Unbelievable 😲. I think they ...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
2,2,2022-10-16 18:01:56+00:00,@DavidPepper People need to be reminded that t...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
3,3,2022-10-15 17:42:10+00:00,Thread. This is my deep fear as well. There ha...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
4,4,2022-10-15 15:15:24+00:00,@Energy_Tidbits @cafreeland Canadian governmen...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
...,...,...,...,...
10731641,14909120,2021-06-16 05:07:52+00:00,"@academy_share Sir, Jyothy Labs Ltd. Share cap...","{'_type': 'snscrape.modules.twitter.User', 'us..."
10731642,14909961,2021-02-03 04:10:45+00:00,@Rawlo__ @rtsmallstreams @share_stream @Stream...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
10731643,14910017,2021-01-25 14:43:02+00:00,Thank you everyone I met my goal of 200 follow...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
10731644,14910021,2021-01-25 05:26:00+00:00,@SharppzTTV @LDTStreamers @BlazedRTs @sme_rt @...,"{'_type': 'snscrape.modules.twitter.User', 'us..."


In [14]:
filtered_df = all_df.iloc[index_list, :]
filtered_df.head

<bound method NDFrame.head of           Unnamed: 0                       date  \
40                40  2022-09-23 01:48:43+00:00   
62                62  2022-09-08 12:01:04+00:00   
63                63  2022-09-08 05:54:58+00:00   
71                71  2022-09-04 19:09:39+00:00   
77                77  2022-09-03 13:47:58+00:00   
...              ...                        ...   
10731579    14897867  2020-01-27 19:53:11+00:00   
10731582    14897870  2020-01-27 16:50:55+00:00   
10731610    14897899  2020-01-15 08:10:13+00:00   
10731611    14897900  2020-01-14 20:50:08+00:00   
10731638    14904570  2022-05-25 05:29:41+00:00   

                                                 rawContent  \
40        Post-Dobbs, Your Private Data Will Be Used Aga...   
62        Why voters get to decide who must give birth? ...   
63        @III_FFC @wttw Mr. Hubert may be right that "f...   
71        @kjblackhawk @JH87712580 @JoyceWhiteVance Wome...   
77        @BlueCollarDreg @Divanista96 @pn

In [16]:
filtered_df.to_csv('/home/karl/PycharmProjects/UCSF_dobbs_roe/data/processed_data/lda_filtered_df_v4.csv')